# Predecir próxima compra de un usuario de MercadoLibre

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import random
from datetime import datetime
from reco import RecoSys
from reco import AutoReco

from gensim.models import Word2Vec 
from gensim.utils import simple_preprocess
from gensim.utils import deaccent
from gensim.models import KeyedVectors
import json
import time

In [4]:
# # Load y_test
# y_test = pd.read_csv('test_data.csv')

# # Load user_search list to train the model
# with open('user_search.txt','r') as infile:
#      user_search = json.load(infile)

# # Load test user search
# with open('user_search_test_void.txt','r') as infile:
#      user_search_test = json.load(infile)

# # Load train words, it is the concatenation of item views titles and unique user searchs
# with open('train_words.txt', 'r') as infile:
#     train_words = json.load(infile)

# Load items view titles
with open("item_title.txt", "r") as infile:
    items_title = json.load(infile)

# # Load item bought id
# with open('bouid_void.txt','r') as infile:
#      bouid = json.load(infile)

# # Load item bought title
# with open('item_bought_void.txt','r') as infile:
#      item_bought = json.load(infile)

# Load metadata 
with open('metadata.txt','r') as infile:
     metadata = json.load(infile)

# Load recommendation test dictionary 
with open('reco_test.txt','r') as infile:
     reco_test = json.load(infile)

In [58]:
# Load recommendation test dictionary 
with open('reco_test_wid.txt','r') as infile:
     reco_test_wid = json.load(infile)

In [5]:
# Load W2V model
model = Word2Vec.load("reco_userview_search_trained.model")

In [155]:
max_sample = 10
auto_rec = AutoReco(max_sample, 0, model, reco_test_wid, items_title)
auto_rec.reco_func()

Loop 10 took 1.674 minutes10 items took 47.256 minutes to find recommendation list


In [163]:
users_recom = auto_rec.reco_list.keys()
users_recom

dict_keys(['18999', '5475', '1119', '20102', '27770', '24850', '23467', '8539', '12582', '17523'])

In [166]:
user_true_bought = [reco_test_wid[user]['item_bought'] for user in users_recom]
del user_true_bought[2]

In [168]:
user_pred = [auto_rec.reco_list[user] for user in users_recom]
del user_pred[2]

In [174]:
def ndcg(y_pred, y_true):
    'Basada en el video de la presentación del challenge'
    dcg = 0
    idcg = 22.42461597 * len(y_true)
    for pred, true in zip(y_pred, y_true):
        position = 1
        for item in pred:
            if item == true:
                dcg += 12 / (np.log(1 + position))
            elif metadata[item]['domain_id'] == metadata[str(true)]['domain_id']:
                dcg += 1 / (np.log(1 + position))
            position += 1
    score = dcg / idcg
    
    return score

In [175]:
ndcg(user_pred, user_true_bought)

0.025689319321634242